<a href="https://colab.research.google.com/github/BeesieJF/BeesieJF/blob/main/MySqlconnector_CloudSQL_GSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Cloud SQL DB Instance

---

This notebook allows you to fill in the Instance, DB name and connects to the GoSmartValue DB Instance on Google Cloud.

In [ ]:
#@title
# install dependencies
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pymysql"] SQLAlchemy==2.0.7
from google.cloud.sql.connector import Connector
import google.auth
from google.auth.transport.requests import Request
import sqlalchemy
import pandas as pd
from sqlalchemy import update
from sqlalchemy.orm import sessionmaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.37.0 whi

In [ ]:
#@title
from google.colab import auth

auth.authenticate_user()

In [ ]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "gosmartvalue-270322" #@param {type:"string"}

assert project_id

!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
#@markdown Please fill in the both the Google Cloud region and name for your Cloud SQL instance. Once filled in, run the cell.

# Please fill in these values.
region = "europe-west1" #@param {type:"string"}
instance_name = "gosmartvalue-db" #@param {type:"string"}

# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("MYSQL"):
  print("Found existing MySQL Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  #password = input("Please provide a password to be used for database 'root' user: ")
  #!gcloud sql instances create {instance_name} --database-version=MYSQL_8_0 \
   # --region={region} --cpu=1 --memory=4GB --root-password={password} \
    #--database-flags=cloudsql_iam_authentication=On

Found existing MySQL Cloud SQL Instance!


In [ ]:
# initialize parameters

#@markdown Please fill in the access credentials for your Cloud SQL instance. Once filled in, run the cell.
INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "Bisi" #@param {type:"string"}
DB_PASS = "@doreG0d" #@param {type:"string"}
DB_NAME = "gosmartvalue" #@param {type:"string"}

Your instance connection name is: gosmartvalue-270322:europe-west1:gosmartvalue-db


In [ ]:
#@markdown Please fill in the query you'd like to run here. Once filled in, run the cell.

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

# connect to connection pool
with pool.connect() as db_conn:

    # store query in a string variable and execute
    #-----UPDATE SECTION
    #SQL_query1 = "UPDATE Baskets SET BuyerId = '08db8f64-2da8-472d-8182-7b9cf532c67d', CreatedBy = '08db8f64-2da8-472d-8182-7b9cf532c67d', UpdatedBy = '08db8f64-2da8-472d-8182-7b9cf532c67d' WHERE Id = '71'"
    #db_conn.execute(sqlalchemy.text(SQL_query1))
    #db_conn.commit()

    #-----FETCH SECTION
    SQL_query = "SELECT * FROM HPITempData" #@param {type:"string"}
    result = db_conn.execute(sqlalchemy.text(SQL_query)).fetchall()
    df = pd.DataFrame(result)
    print(df.tail(50)) #output the top 50 rows of the table

    df.to_csv("ScrappedData.csv", index=False) #, columns=['Category', 'Type', 'SalePrice', 'ScrapDateTime'])

    #results.to_csv(os.path.join(basedir, 'mydump2.csv'),index=False,sep=";")

# cleanup connector object

connector.close()

                                       Id HPIWebsiteId HPIWebsiteName  \
322  08dd1d3b-27b3-4903-8dcd-33188905f50d         None     Property24   
323  08dd1d3b-27b3-4917-8edf-836e5b1e204b         None     Property24   
324  08dd1d3b-27b3-492b-8242-a7480e3940be         None     Property24   
325  08dd1d3b-27b3-494a-8ce4-d6c98d0a9906         None     Property24   
326  08dd1d3b-27b3-496a-80ff-f41e455da0ac         None     Property24   
327  08dd1d3b-27b3-4987-86da-78c8b483dbd8         None     Property24   
328  08dd1d3b-27b3-49ac-8ed2-5e3afa673133         None     Property24   
329  08dd1d3b-27b3-49d3-857b-38c7b7399d59         None     Property24   
330  08dd1d3b-27b3-49f7-8132-620050f9ac21         None     Property24   
331  08dd1d3b-27b3-4a0f-810f-5634cbe412b3         None     Property24   
332  08dd1d3b-27b3-4a2b-8fb4-13ca749a249b         None     Property24   
333  08dd1d3b-27b3-4a38-803f-3e1694abcf0d         None     Property24   
334  08dd1d3b-27b3-4a42-85f6-2b0417ae3233         N

In [ ]:
from google.colab import drive

drive.mount('/content/drive') #/Rental Rates - Sheet1.csv’)

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/Amenities data Collection sheet  - Amenities.csv'
df = pd.read_csv(file_path)

#df.dropna()
#df.dropna(axis=0, subset=[['Location ID', 'LOcality ID']])
df

,Location ID,Locality ID,Name,Type,Latitude,Longitude,Plot size (sq.m.)
0,27.0,112.0,"ZCC, Pentacostal",Place of Worship,"21°11'0.08""S","27°30'49.60""E",23624.0
1,27.0,112.0,Pentacostal Holiness,Place of Worship,"21°11'4.56""S","27°30'60.00""E",2493.0
2,27.0,112.0,Phase 4 Primary,School,"21°11'16.02""S","27°31'10.74""E",48532.0
3,27.0,1.0,Block 2 Shops,Shops,"21°11'31.57""S","27°31'7.35""E",3493.0
4,27.0,2.0,Phatsimo Primary,School,"21°11'43.91""S","27°31'9.84""E",29956.0
...,...,...,...,...,...,...,...
923,NaN,NaN,Erbn Club,Night Club,"24°39'35.37""S","25°32'16.36""E",2233.0
924,NaN,NaN,Saverite,Shopping Complex,"24°39'25.84""S","25°32'13.33""E",1839.0
925,NaN,NaN,NaN,Guest House,"24°39'23.37""S","25°32'6.54""E",4440.0
926,NaN,NaN,Heroes Park,General Dealer,"24°39'21.87""S","25°32'8.80""E",NaN


In [ ]:
df.dtypes

Location ID          float64
Locality ID          float64
Name                  object
Type                  object
Latitude              object
Longitude             object
Plot size (sq.m.)    float64
dtype: object

In [ ]:
df = df.dropna(axis=0, subset=['Location ID', 'Locality ID '])

In [ ]:
df.fillna('UNAVAILABLE', inplace=True)

<ipython-input-110-773b0eef4777>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna('UNAVAILABLE', inplace=True)


In [ ]:
df.head(50)

,Location ID,Locality ID,Name,Type,Latitude,Longitude,Plot size (sq.m.)
0,27.0,112.0,"ZCC, Pentacostal",Place of Worship,"21°11'0.08""S","27°30'49.60""E",23624.0
1,27.0,112.0,Pentacostal Holiness,Place of Worship,"21°11'4.56""S","27°30'60.00""E",2493.0
2,27.0,112.0,Phase 4 Primary,School,"21°11'16.02""S","27°31'10.74""E",48532.0
3,27.0,1.0,Block 2 Shops,Shops,"21°11'31.57""S","27°31'7.35""E",3493.0
4,27.0,2.0,Phatsimo Primary,School,"21°11'43.91""S","27°31'9.84""E",29956.0
5,27.0,2.0,The Nest Lodge,Hotel/Conferencing,"21°11'38.10""S","27°30'46.29""E",1834.0
6,27.0,2.0,Wingate Guest House,Guest House,"21°11'40.99""S","27°30'46.33""E",2128.0
7,27.0,2.0,Morning Star,Guest House,"21°11'42.00""S","27°30'49.92""E",400.0
8,27.0,2.0,Olo's Cottage,Guest House,"21°11'44.14""S","27°30'48.95""E",400.0
9,27.0,2.0,Prestige Hotel,Hotel/Conferencing,"21°11'45.37""S","27°30'46.66""E",2051.0


In [ ]:
df['Location ID'] = df['Location ID'].astype(int)

<ipython-input-112-3cbdf20bbaa2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Location ID'] = df['Location ID'].astype(int)


In [ ]:
df.dtypes

Location ID            int64
Locality ID          float64
Name                  object
Type                  object
Latitude              object
Longitude             object
Plot size (sq.m.)     object
dtype: object

In [ ]:
#df
df = df.astype({"Locality ID ":'int'})

In [ ]:
df.dtypes

Location ID           int64
Locality ID           int64
Name                 object
Type                 object
Latitude             object
Longitude            object
Plot size (sq.m.)    object
dtype: object

In [ ]:
df.head(50)

,Location ID,Locality ID,Name,Type,Latitude,Longitude,Plot size (sq.m.)
0,27,112,"ZCC, Pentacostal",Place of Worship,"21°11'0.08""S","27°30'49.60""E",23624.0
1,27,112,Pentacostal Holiness,Place of Worship,"21°11'4.56""S","27°30'60.00""E",2493.0
2,27,112,Phase 4 Primary,School,"21°11'16.02""S","27°31'10.74""E",48532.0
3,27,1,Block 2 Shops,Shops,"21°11'31.57""S","27°31'7.35""E",3493.0
4,27,2,Phatsimo Primary,School,"21°11'43.91""S","27°31'9.84""E",29956.0
5,27,2,The Nest Lodge,Hotel/Conferencing,"21°11'38.10""S","27°30'46.29""E",1834.0
6,27,2,Wingate Guest House,Guest House,"21°11'40.99""S","27°30'46.33""E",2128.0
7,27,2,Morning Star,Guest House,"21°11'42.00""S","27°30'49.92""E",400.0
8,27,2,Olo's Cottage,Guest House,"21°11'44.14""S","27°30'48.95""E",400.0
9,27,2,Prestige Hotel,Hotel/Conferencing,"21°11'45.37""S","27°30'46.66""E",2051.0


In [ ]:
#df.rename(columns= {'1 bed `: '1 bed', `2 bed`: '2 bed'},inplace=True)
df = df.rename(columns={'Longitude ': 'Longitude'}) #, 'oldName2': 'newName2'})
#df.replace(0.0, '', inplace=True)
#df

In [ ]:
# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

# connect to connection pool
#with pool.connect() as db_conn:
    # store query in a string variable and execute
    #SQL_query = "SELECT * FROM RentalRates" #@param {type:"string"}
    #result = db_conn.execute(sqlalchemy.text(SQL_query)).fetchall()
    #df = pd.DataFrame(result)
    #print(df) #output the top 10 rows of the table

    #df.to_sql('RentalRates', con=db_conn, index=True, index_label='id', if_exists='replace')
#df.to_sql('Amenities', con=pool, index=False, if_exists='replace')
con=pool.connect()
#query = 'SELECT * FROM ReportRequests'
df = pd.read_sql_table('ComparableResults', con)

#df.dtypes

# cleanup connector object
connector.close()


In [ ]:
df.dtypes

ReferenceNumber                     object
ValuationId                         object
EstimatedValue                     float64
DeletedOn                   datetime64[ns]
EstimatedOn                 datetime64[ns]
IsDeleted                            int64
BathRooms                            int64
BedRooms                             int64
Email                               object
FirstName                           object
Garages                              int64
Kitchens                             int64
LastName                            object
MobileNumber                        object
ReportType                          object
SittingRooms                         int64
Toilets                              int64
RequesterUserId                     object
ComparableId                        object
PaymentStatus                        int64
ReportRequested                      int64
BoundaryWall                         int64
ElectricFence                        int64
FirePlace  